<a href="https://colab.research.google.com/github/h2o4sure01/100DaysofML/blob/master/Advanced_Ensemble_Method_Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Ensemble Method: Stacking

After spending about a lot of time on Data Analysis. You know all the EDAs, feature engineering and all other cool stuffs we do in Data analysis,  you would not want to waste time on model training, you probably just to spend little time 

##Now lets import the neccesary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

The datasets are loaded from google drive, we have the train data set which will be use to train the model, the test dataset in which the model will be test on and the sample submission file. 

In [ ]:
train = pd.read_csv('/content/drive/My Drive/bootcamp/Train.csv')
test = pd.read_csv('/content/drive/My Drive/bootcamp/Test.csv')
s = pd.read_csv('/content/drive/My Drive/bootcamp/SampleSubmissio.csv')

In [ ]:
#print the first five row of the train set
train.head(5)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [ ]:
train= train.drop('Applicant_ID',1)
test = test.drop('Applicant_ID',1)

We can can see from the table that there are 52 columns including the target varaible. We alo notice some column that contain NaN so we know we'll surely get our hands dirty. But before that lets check the shape of the datasets

In [ ]:
train.shape, test.shape, s.shape

((56000, 51), (24000, 50), (24000, 2))

So now we know that our train set has 56000 rows, 51 coulmuns while test set has 24000 rows and 50 columns excluding the target vairaible (default_status)
lets check for some info on the dataset

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 51 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   form_field1     53471 non-null  float64
 1   form_field2     52156 non-null  float64
 2   form_field3     55645 non-null  float64
 3   form_field4     55645 non-null  float64
 4   form_field5     55645 non-null  float64
 5   form_field6     42640 non-null  float64
 6   form_field7     50837 non-null  float64
 7   form_field8     42640 non-null  float64
 8   form_field9     47992 non-null  float64
 9   form_field10    55645 non-null  float64
 10  form_field11    24579 non-null  float64
 11  form_field12    46105 non-null  float64
 12  form_field13    50111 non-null  float64
 13  form_field14    56000 non-null  int64  
 14  form_field15    33525 non-null  float64
 15  form_field16    42964 non-null  float64
 16  form_field17    44849 non-null  float64
 17  form_field18    45598 non-null 

From the output above we could see that we have some columns that are objects (categorical columns) and the rest are floats and integer. We need to convert those categorical coulumns to numeric.

In [ ]:
from sklearn.preprocessing import LabelBinarizer
l = LabelBinarizer()
train['form_field47'] = l.fit_transform(train['form_field47'])
test['form_field47'] = l.fit_transform(test['form_field47'])
train['default_status'] = l.fit_transform(train['default_status'])

In [ ]:
#let check 
train.head(3)

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,0,NaN,1.129518,0.044335,0
1,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,0,349.80573,1.620483,0.322436,0
2,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,0,NaN,1.513370,0.011640,1


We have successfully covert the categorial columns to numeric.

The next the we need to do is to check for missing values.

In [ ]:
train.isnull().sum()

form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41      38229
form_field42    

In [ ]:
test.isnull().sum()

form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field45    13538
form_field

Most of the columns have missing values, in fact there are columns with over 30000 NaN. There are several ways of dealing with nan is a dataset but we are going to fill them with -99999 in this case

In [ ]:
train.fillna(-9999, inplace=True)
test.fillna(-9999,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
#x = train.drop('default_status',1)
#y = train.default_status

#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=42)

In [ ]:
x_columns = train.columns.drop('default_status')
x = train[x_columns].values
y = train['default_status'].values # Classification
x_submit = test[x_columns].values.astype(np.float32)


In [ ]:
!pip install catboost

     |████████████████████████████████| 66.1MB 72kB/s 


In [ ]:

from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

SHUFFLE = False
FOLDS = 10


def mlogloss(y_test, preds):
    epsilon = 1e-15
    sum = 0
    for row in zip(preds,y_test):
        x = row[0][row[1]]
        x = max(epsilon,x)
        x = min(1-epsilon,x)
        sum+=math.log(x)
    return( (-1/len(preds))*sum)

def stretch(y):
    return (y - y.min()) / (y.max() - y.min())


def blend_ensemble(x, y, x_submit):
    kf = StratifiedKFold(FOLDS)
    folds = list(kf.split(x,y))

    models = [
        XGBClassifier(max_depth=10, learning_rate=0.1),
                      
        CatBoostClassifier(max_depth=10, learning_rate=0.1),
        LGBMClassifier(max_depth=10, learning_rate=0.1)
        ]
                            
                            
                            

    dataset_blend_train = np.zeros((x.shape[0], len(models)))
    dataset_blend_test = np.zeros((x_submit.shape[0], len(models)))

    for j, model in enumerate(models):
        print("Model: {} : {}".format(j, model) )
        fold_sums = np.zeros((x_submit.shape[0], len(folds)))
        total_loss = 0
        for i, (train, test) in enumerate(folds):
            x_train = x[train]
            y_train = y[train]
            x_test = x[test]
            y_test = y[test]
            model.fit(x_train, y_train)
            pred = np.array(model.predict_proba(x_test))
            # pred = model.predict_proba(x_test)
            dataset_blend_train[test, j] = pred[:, 1]
            pred2 = np.array(model.predict_proba(x_submit))
            #fold_sums[:, i] = model.predict_proba(x_submit)[:, 1]
            fold_sums[:, i] = pred2[:, 1]
            loss = mlogloss(y_test, pred)
            total_loss+=loss
            print("Fold #{}: loss={}".format(i,loss))
        print("{}: Mean loss={}".format(model.__class__.__name__,
                                        total_loss/len(folds)))
        dataset_blend_test[:, j] = fold_sums.mean(1)

    print()
    print("Blending models.")
    blend = RandomForestClassifier()
    blend.fit(dataset_blend_train, y)
    
    return blend.predict_proba(dataset_blend_test)

if __name__ == '__main__':

    np.random.seed(42)  # seed to shuffle the train set

  

    predictors = list(train.columns.values)
    predictors.remove('default_status')
    x = train[predictors].values
    y = train['default_status']
    x_submit = test.values

    if SHUFFLE:
        idx = np.random.permutation(y.size)
        x = x[idx]
        y = y[idx]

    submit_data = blend_ensemble(x, y, x_submit)
    submit_data = stretch(submit_data)

Streaming output truncated to the last 5000 lines.
26:	learn: 0.3887428	total: 4.21s	remaining: 2m 31s
27:	learn: 0.3874347	total: 4.37s	remaining: 2m 31s
28:	learn: 0.3856693	total: 4.54s	remaining: 2m 31s
29:	learn: 0.3846682	total: 4.69s	remaining: 2m 31s
30:	learn: 0.3836647	total: 4.85s	remaining: 2m 31s
31:	learn: 0.3827696	total: 5s	remaining: 2m 31s
32:	learn: 0.3814275	total: 5.15s	remaining: 2m 30s
33:	learn: 0.3807002	total: 5.31s	remaining: 2m 30s
34:	learn: 0.3799626	total: 5.46s	remaining: 2m 30s
35:	learn: 0.3787750	total: 5.62s	remaining: 2m 30s
36:	learn: 0.3776712	total: 5.78s	remaining: 2m 30s
37:	learn: 0.3764743	total: 5.93s	remaining: 2m 30s
38:	learn: 0.3753132	total: 6.09s	remaining: 2m 30s
39:	learn: 0.3745472	total: 6.24s	remaining: 2m 29s
40:	learn: 0.3733622	total: 6.4s	remaining: 2m 29s
41:	learn: 0.3726856	total: 6.55s	remaining: 2m 29s
42:	learn: 0.3714250	total: 6.72s	remaining: 2m 29s
43:	learn: 0.3704132	total: 6.88s	remaining: 2m 29s
44:	learn: 0.3698

In [ ]:
submit_df = pd.DataFrame({'Applicant_ID':s.Applicant_ID, 'default_status':submit_data[:, 1]})
submit_df.to_csv('submit_filename7.csv', index=False)

In [ ]:
from sklearn import metrics
import scipy as sp
import numpy as np
import math
from sklearn import metrics


def perturbation_rank(model, x, y, names, regression):
    errors = []

    for i in range(x.shape[1]):
        hold = np.array(x[:, i])
        np.random.shuffle(x[:, i])
        
        if regression:
            pred = model.predict(x)
            error = metrics.mean_squared_error(y, pred)
        else:
            pred = model.predict_proba(x)
            error = metrics.log_loss(y, pred)
            
        errors.append(error)
        x[:, i] = hold
        
    max_error = np.max(errors)
    importance = [e/max_error for e in errors]

    data = {'name':names,'error':errors,'importance':importance}
    result = pd.DataFrame(data, columns = ['name','error','importance'])
    result.sort_values(by=['importance'], ascending=[0], inplace=True)
    result.reset_index(inplace=True, drop=True)
    return result